# 상담사챗봇

## 프로젝트 목적

* 상담사 챗봇을 통해 인건비를 줄일 수 있다.
* 상담사가 답변하지 못하는 답변을 할 수 있다.
* 언제 어디서는 상담을 받을 수 있어 편리하다.

## 프로젝트 개요

* AI허브에 있는 민원 질의응답 데이터 사용(https://aihub.or.kr/aidata/30716)
* sentencepiece와 Transformer 모델을 사용하여 챗봇을 구현


## 프로젝트 내용

* AI허브에 있는 데이터중 금융보험 데이터만 사용
* json을 파싱하여 필요한 데이터만 추출
* Transformer와 embedding 모델을 사용하여 학습

### 필요한 모듈 import

* 데이터셋: https://aihub.or.kr/aidata/30716

In [ ]:
import pandas as pd
import numpy as np
import re
import json
import os
import sentencepiece as spm
from torch.utils.data import Dataset, DataLoader
from torch.nn import Transformer
from torch import nn
import torch
import math

### 데이터 로드 및 전처리

In [ ]:
# json파싱후 불필요한 데이터 삭제
dir_path = "/home/dilab05/work_directory/adj/NLP_Project/Training/금융보험/민원(콜센터) 질의응답_금융보험_잔고 및 거래내역_Training.json"
with open(dir_path, 'r') as file:
    li = []
    data = json.load(file)
    for i in range(len(data)):
        del(data[i]['도메인'])
        del(data[i]['카테고리'])
        del(data[i]['대화셋일련번호'])
        del(data[i]['화자'])
        del(data[i]['문장번호'])
        del(data[i]['고객의도'])
        del(data[i]['상담사의도'])
        del(data[i]['QA'])
        del(data[i]['개체명 '])
        del(data[i]['용어사전'])
        del(data[i]['지식베이스'])
        del(data[i]['상담사질문(요청)'])
        del(data[i]['고객답변'])
    for i in range(len(data)):    # 84031
        if (data[i]['고객질문(요청)'] == '') and (data[i]['상담사답변'] == ''):
            del data[i]['고객질문(요청)']
            del data[i]['상담사답변']
        else:
            if data[i]['고객질문(요청)'] == '':
                del data[i]['고객질문(요청)']
            elif data[i]['상담사답변'] == '':
                del data[i]['상담사답변']
            else:
                del data[i]['고객질문(요청)']
                del data[i]['상담사답변']
            li.append(data[i])

In [ ]:
# 고객 질문 다음 상담사 답변이 오는 것만 추출
idxa = []
idxq = []
for i in range(len(li)):
    if '고객질문(요청)' in li[i] and '상담사답변' in li[i+1]:
        idxq.append(i)

In [ ]:
# 매핑하기 위해 넘파이 배열 선언
li = np.array(li)
idxa = np.array(idxa)
idxq = np.array(idxq)

print(li)
print(idxa)
print(idxq)

In [ ]:
# 데이터 매핑
q = li[idxq]
a = li[idxq+1]

#question = []
#answer = []
for i in range(len(q)):
    question.append(q[i]['고객질문(요청)'])
for i in range(len(q)):
    answer.append(a[i]['상담사답변'])

In [ ]:
# 넘파이 배열로 저장
#np.savez_compressed('./data', question=question, answer=answer)

In [ ]:
# 데이터 로드
question = np.load('./data.npz')['question']
answer = np.load('./data.npz')['answer']

In [ ]:
# 판다스 데이터 프레임 형식으로 저장
train_data = pd.DataFrame([ x for x in zip(question, answer)], columns = ['Q','A'])
train_data

In [ ]:
# 공백제거
q_null = train_data[train_data['Q'] == ' '].index
train_data = train_data.drop(q_null)
a_null = train_data[train_data['A'] == ' '].index
train_data = train_data.drop(a_null)

In [ ]:
# 불필요한 특수문자 제거
questions = []
for sentence in train_data['Q']:
	# 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,~])", r"", sentence)
    sentence = sentence.strip()
    questions.append(sentence)

answers = []
for sentence in train_data['A']:
    sentence = re.sub(r"([?.!,~])", r"", sentence)
    sentence = sentence.strip()
    answers.append(sentence)

In [ ]:
# 질문과 답변을 순서대로 txt파일로 저장
with open('data.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(questions))
    f.write('\n'.join(answers))

In [ ]:
# sentencepiece를 사용하여 8000개의 vocab size를 가지고 사용자 지정 토큰 7개를 추가로 가지고 있는 sentence piece를 학습
corpus = "data.txt"
prefix = "chatbot"
vocab_size = 8000
spm.SentencePieceTrainer.train(
    f"--input={corpus} --model_prefix={prefix} --vocab_size={vocab_size + 7}" + 
    " --model_type=bpe" +
    " --max_sentence_length=999999" + # 문장 최대 길이
    " --pad_id=0 --pad_piece=[PAD]" + # pad (0)
    " --unk_id=1 --unk_piece=[UNK]" + # unknown (1)
    " --bos_id=2 --bos_piece=[BOS]" + # begin of sequence (2)
    " --eos_id=3 --eos_piece=[EOS]" + # end of sequence (3)
    " --user_defined_symbols=[SEP],[CLS],[MASK]") # 사용자 정의 토큰

In [ ]:
# 토큰화 테스트
vocab_file = "chatbot.model"
vocab = spm.SentencePieceProcessor()
vocab.load(vocab_file)
line = "무엇을 도와드릴까요?"
pieces = vocab.encode_as_pieces(line)
ids = vocab.encode_as_ids(line)


print(line)
print(pieces)

### 모델 구현


In [ ]:
# 학습된 sentence piece를 이용하여 주어진 문장을 정수로 인코딩하는 함수를 선언. 
# 문장의 처음과 끝에는 sentence piece를 학습 시킬 때 따로 선언했던 START_TOKEN과 END_TOKEN의 index를 넣음
MAX_LENGTH = 40

START_TOKEN = [2]
END_TOKEN = [3]

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
    tokenized_inputs, tokenized_outputs = [], []

    for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
        zeros1 = np.zeros(MAX_LENGTH, dtype=int)
        zeros2 = np.zeros(MAX_LENGTH, dtype=int)
        sentence1 = START_TOKEN + vocab.encode_as_ids(sentence1) + END_TOKEN
        zeros1[:len(sentence1)] = sentence1[:MAX_LENGTH]

        sentence2 = START_TOKEN + vocab.encode_as_ids(sentence2) + END_TOKEN
        zeros2[:len(sentence2)] = sentence2[:MAX_LENGTH]

        tokenized_inputs.append(zeros1)
        tokenized_outputs.append(zeros2)
    return tokenized_inputs, tokenized_outputs

In [ ]:
# 인코딩 테스트
questions_encode, answers_encode = tokenize_and_filter(questions, answers)
print(questions_encode[1])
print(answers_encode[1])

In [ ]:
# Batch 학습을 위해 dataset, dataloader를 생성
# 첫번째 값은 주어진 질문, 두번째 값은 디코더의 입력으로 마지막 토큰값이 제거된 대답, 마지막 값은 첫 토큰값이 제거된 결과
class SequenceDataset(Dataset):
    def __init__(self, questions, answers):
        questions = np.array(questions)
        answers = np.array(answers)
        self.inputs = questions
        self.dec_inputs = answers[:,:-1]
        self.outputs = answers[:,1:]
        self.length = len(questions)
    
    def __getitem__(self,idx):
        return (self.inputs[idx], self.dec_inputs[idx], self.outputs[idx])

    def __len__(self):
        return self.length

BATCH_SIZE = 64
dataset = SequenceDataset(questions_encode, answers_encode)
dataloader = DataLoader(dataset, shuffle=True, batch_size=BATCH_SIZE)

In [ ]:
# Transformer 모델 선언(positional encoding과 Embedding이 포함)
class TFModel(nn.Module):
    def __init__(self, ntoken, ninp, nhead, nhid, nlayers, dropout=0.5):
        super(TFModel, self).__init__()
        self.transformer = Transformer(ninp, nhead, dim_feedforward=nhid, num_encoder_layers=nlayers, num_decoder_layers=nlayers,dropout=dropout)
        self.pos_encoder = PositionalEncoding(ninp, dropout)
        self.encoder = nn.Embedding(ntoken, ninp)

        self.pos_encoder_d = PositionalEncoding(ninp, dropout)
        self.encoder_d = nn.Embedding(ntoken, ninp)

        self.ninp = ninp
        self.ntoken = ntoken

        self.linear = nn.Linear(ninp, ntoken)
        self.init_weights()

    def generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def init_weights(self):
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src, tgt, srcmask, tgtmask, srcpadmask, tgtpadmask):
        src = self.encoder(src) * math.sqrt(self.ninp)
        src = self.pos_encoder(src)

        tgt = self.encoder_d(tgt) * math.sqrt(self.ninp)
        tgt = self.pos_encoder_d(tgt)


        output = self.transformer(src.transpose(0,1), tgt.transpose(0,1), srcmask, tgtmask, src_key_padding_mask=srcpadmask, tgt_key_padding_mask=tgtpadmask)
        output = self.linear(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

def gen_attention_mask(x):
    mask = torch.eq(x, 0)
    return mask

### 모델 학습 및 평가

In [ ]:
# 모델학습
device = torch.device("cuda", index=1)

lr = 1e-4
model = TFModel(vocab_size+7, 256, 8, 512, 2, 0.1).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
epoch = 20
best_loss = 2000.0

from tqdm import tqdm

model.load_state_dict(torch.load("chatbot.pth"))
model.train()
for i in range(epoch):
    batchloss = 0.0
    progress = tqdm(dataloader)
    for (inputs, dec_inputs, outputs) in progress:
        optimizer.zero_grad()
        src_mask = model.generate_square_subsequent_mask(MAX_LENGTH).to(device)
        src_padding_mask = gen_attention_mask(inputs).to(device)
        tgt_mask = model.generate_square_subsequent_mask(MAX_LENGTH-1).to(device)
        tgt_padding_mask = gen_attention_mask(dec_inputs).to(device)

        result = model(inputs.to(device), dec_inputs.to(device), src_mask, tgt_mask, src_padding_mask,tgt_padding_mask)
        loss = criterion(result.permute(1,2,0), outputs.to(device).long())
        progress.set_description("{:0.3f}".format(loss))
        loss.backward()
        optimizer.step()
        batchloss += loss
    print(loss)
    print(batchloss)
    if best_loss > batchloss:
        print(f"Best loss: {best_loss}, Loss: {batchloss}")
        best_loss = batchloss
        torch.save(model.state_dict(), "chatbot.pth")
            
    print("epoch:",i+1,"|","loss:",batchloss.cpu().item() / len(dataloader))

In [ ]:
# 모델 평가
def preprocess_sentence(sentence):
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    return sentence

def evaluate(sentence):
    sentence = preprocess_sentence(sentence)
    input = torch.tensor([START_TOKEN + vocab.encode_as_ids(sentence) + END_TOKEN]).to(device)
    output = torch.tensor([START_TOKEN]).to(device)

    # 디코더의 예측 시작
    model.eval()
    for i in range(MAX_LENGTH):
        src_mask = model.generate_square_subsequent_mask(input.shape[1]).to(device)
        tgt_mask = model.generate_square_subsequent_mask(output.shape[1]).to(device)

        src_padding_mask = gen_attention_mask(input).to(device)
        tgt_padding_mask = gen_attention_mask(output).to(device)

        predictions = model(input, output, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask).transpose(0,1)
        # 현재(마지막) 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = torch.LongTensor(torch.argmax(predictions.cpu(), axis=-1))


        # 만약 마지막 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if torch.equal(predicted_id[0][0], torch.tensor(END_TOKEN[0])):
            break

        # 마지막 시점의 예측 단어를 출력에 연결한다.
        # 이는 for문을 통해서 디코더의 입력으로 사용될 예정이다.
        output = torch.cat([output, predicted_id.to(device)], axis=1)

    return torch.squeeze(output, axis=0).cpu().numpy()

def predict(sentence):
    prediction = evaluate(sentence)
    predicted_sentence = vocab.Decode(list(map(int,[i for i in prediction if i < vocab_size+7])))

    print('Input: {}'.format(sentence))
    print('Output: {}'.format(predicted_sentence))

    return predicted_sentence

### 결과 및 응용

In [ ]:
model.load_state_dict(torch.load("chatbot.pth"))
result = predict("보험적용이 되나요?")

## 참고 문헌

* sentencepiece: https://wikidocs.net/86657
